# comercio_ext_auxiliares.tb_referencia_pais_bloco
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/PAIS_BLOCO_delta`
## 📌 Descrição do arquivo
Referência de **Blocos Econômicos por País**.
|Coluna|Descrição|
|---|---|
|`CO_PAIS`|Código do país|
|`CO_BLOCO`|Código do bloco|
|`NO_BLOCO`|Nome do bloco (PT)|
|`NO_BLOCO_ING`|Nome do bloco (EN)|
|`NO_BLOCO_ESP`|Nome do bloco (ES)|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
def up(c): return F.upper(F.trim(F.col(c).cast(T.StringType())))

In [0]:
bronzePath  = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/pais_bloco/"
silverTable = "silver_comercio_ext_auxiliares.tb_referencia_pais_bloco"

In [0]:


silverSchema = T.StructType([
    T.StructField("CO_PAIS",     T.StringType(),   False),
    T.StructField("CO_BLOCO",    T.StringType(),   False),
    T.StructField("NO_BLOCO",    T.StringType(),   False),
    T.StructField("NO_BLOCO_ING",T.StringType(),   True ),
    T.StructField("NO_BLOCO_ESP",T.StringType(),   True ),
    T.StructField("TS_REF",      T.TimestampType(),False),
    T.StructField("NM_ORIGEM",   T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:


df_bronze = spark.read.format("delta").load(bronzePath)



##Normalização
> #### **datatype**

In [0]:
df_norm = (
    df_bronze
    .withColumn("CO_PAIS",  up("CO_PAIS"))
    .withColumn("CO_BLOCO", up("CO_BLOCO"))
    .withColumn("NO_BLOCO", F.col("NO_BLOCO").cast(T.StringType()))
    .withColumn("NO_BLOCO_ING", F.col("NO_BLOCO_ING").cast(T.StringType()))
    .withColumn("NO_BLOCO_ESP", F.col("NO_BLOCO_ESP").cast(T.StringType()))
    .withColumn("TS_REF",   F.current_timestamp())
    .withColumn("NM_ORIGEM",F.lit("/landingbeca2026jan/balancacomercial/pais_bloco"))
)



##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_PAIS").isNotNull() & F.col("CO_BLOCO").isNotNull())


In [0]:
df_dedup = df_valid.dropDuplicates(["CO_PAIS","CO_BLOCO"])



In [0]:
df_silver = df_dedup.select(
    "CO_PAIS","CO_BLOCO","NO_BLOCO","NO_BLOCO_ING","NO_BLOCO_ESP","TS_REF","NM_ORIGEM"
)

##Carga
> #### **overwrite**

In [0]:
delta_target = DeltaTable.forName(spark, silverTable)
merge_condition = "t.CO_PAIS = s.CO_PAIS AND t.CO_BLOCO = s.CO_BLOCO"

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "NO_BLOCO":    "s.NO_BLOCO",
        "NO_BLOCO_ING":"s.NO_BLOCO_ING",
        "NO_BLOCO_ESP":"s.NO_BLOCO_ESP",
        "TS_REF":      "s.TS_REF",
        "NM_ORIGEM":   "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute())